In [222]:
# Initial tests for generating valid grids
def grid(A):
    # choosing A
    third = int(A/3)+1

    nums = list(range(1,A))
    valids = 0

    # running through the combinations
    for g in itertools.permutations(nums, 4):
        # generating guess list
        guesses = {k:0 for k in range(1,A)}
        
        # creating matrix 
        M = np.zeros((3,3))
        
        # assigning the corners
        M[0,0] = g[0]
        M[0,2] = g[1]
        M[2,0] = g[2]
        M[2,2] = g[3]

        # checking center rows
        # ensuring the center values agree
        cl = int(A - (M[0,0]+M[2,2])) 
        cr = int(A - (M[0,2]+M[2,0])) 
        # if c is within range and has not been assigned
        if cr == cl and cl in guesses.keys() and guesses[cl] == 0:
            M[1,1] = cl
            guesses[cl] = 1
        else: continue # otherwise the solution is invalid and the program continues looking for a solution

        # checking top/bottom rows
        t = int(A - (M[0,0]+M[0,2])) # top
        if t in guesses.keys() and guesses[t] == 0:
            M[0,1] = t
            guesses[t] = 1
        else: continue
        
        b = int(A - (M[2,0]+M[2,2])) # bottom
        if b in guesses.keys() and guesses[b] == 0:
            M[2,1] = b
            guesses[b] = 1
        else: continue

        # checking left/right
        l = int(A - (M[0,0]+M[2,0])) # left
        if l in guesses.keys() and guesses[l] == 0:
            M[1,0] = l
            guesses[l] = 1
        else: continue

        r = int(A - (M[0,2]+M[2,2])) # right
        if r in guesses.keys() and guesses[r] == 0:
            M[1,2] = r
            guesses[r] = 1
        else: continue
        # print(M)

        # checking the centre-cross
        tbsum = M[:,1].sum()
        if tbsum not in list(range(A-1,A+2)): continue

        lrsum = M[1,:].sum()
        if lrsum not in list(range(A-1,A+2)): continue

        valids += 1
        display(M)
    # 96 total range

In [ ]:
grid(27)

In [45]:
def equation(A):
    # a third 
    third = A/3
    # list of values
    ns = {x:0 for x in range(int(-1*third), int(2*third))}
    positive = 0
    negative = 0 

    for n in itertools.combinations(list(ns.keys()), 2):
        find = -1*(n[0]+n[1])
        if find in ns.keys():
            if find < 0: negative += 1
            if find > 0: positive += 1
            # print([i for i in n]+[find])
    print(positive/negative)
            

In [138]:
# checking for duplicates in the list
def dupli(As:list) -> bool:
    for A in As:
        if As.count(A) > 1: return True
    return False

In [191]:
A = [1,2,3]
tuple(A)

(1, 2, 3)

In [192]:
d = {'car':4,'bees':3}
if 'car' in d: print(d['car'])

4


In [514]:
def permutationengine(A:tuple, attempted:dict) -> None:
    # going off the base assumption A1<A2<A3<A4
    i = 0
    # brute-forcing the equations, optimizing range selection
    # running the 
    for A12 in range(1,A[0]):
        # creating upper and lower bounds for A24
        u24 = A[1]-A12
        l24 = A[1]-A[0]-A12
        for A24 in range(max(1,l24+1), min(A[1],u24)):
            # upper and lower bounds for A34
            u34 = A[3]-A24
            l34 = A[3]-A[1]-A24
            for A34 in range(max(1,l34+1), min(A[2],u34)):
                # upper and lower for A13
                u13 = A[2]-A34
                for A13 in range(1,min(u13, A[0])):
                    # ensuring only unique values exist
                    current = (A12, A24, A13, A34)
                    if dupli(current): continue
                    
                    # ensuring we are not attempting the same values twice
                    if current not in attempted:
                        attempted[current] = None
                    else: continue

                    # solving for each equation
                    #            eq1               eq2               eq3               eq4
                    eqs = [ A[1] - (A12+A24) , A[3] - (A34+A24) , A[2] - (A34+A13) , A[0] - (A13+A12) ]
                    
                    # legacy code, but rather check than be sorry
                    # checking if equations fall in range 0 < eq_n < min(A_i, A_j) and are all unique
                    if 0 < eqs[0] < A[0] and 0 < eqs[1] < A[1] and 0 < eqs[2] < A[2] and 0 < eqs[3] < A[0] and not dupli(eqs):
                        # storing as {(values of A) : [(free, middle, edge), ... ]} 
                        #                   I12                 I24                 I34                 I13
                        valideqs = [(eqs[0], A12, A24), (eqs[1], A24, A34), (eqs[2], A34, A13), (eqs[3], A13, A12)]
                        propagate(A, valideqs)
                    i+=1
                    if i % 1E6 == 0: print(f"Current A:{A}, Combinations Processed: {i}")

In [515]:
def propagate(A:tuple, valideqs:list):
    # creating matrices 
    A1 = np.zeros((3,3))
    A2 = np.zeros((3,3))
    A3 = np.zeros((3,3))
    A4 = np.zeros((3,3))

    # assigning values from equations to matrices
    # I12
    A1[0,1] = valideqs[0][0]
    A2[2,0] = valideqs[0][0]

    A1[0,2] = valideqs[0][1]
    A2[2,1] = valideqs[0][1]
    
    A2[2,2] = valideqs[0][2]
    A4[1,0] = valideqs[0][2]

    # I24
    A4[0,0] = valideqs[1][0]
    A2[1,2] = valideqs[1][0]
    # middle has already been assigned
    A4[2,0] = valideqs[1][2]
    A3[0,1] = valideqs[1][2]

    # I34
    A3[0,2] = valideqs[2][0]
    A4[2,1] = valideqs[2][0]
    # middle has already been assigned
    A3[0,0] = valideqs[2][2]
    A1[1,2] = valideqs[2][2]
    
    # I13
    A1[2,2] = valideqs[3][0]
    A3[1,0] = valideqs[3][0]
    # middle has already been assigned
    # edge has already been assigned

    # populating matrices, ensuring that all are oriented in the same direction as A1
    Lov = {valideqs[0][0]:None,valideqs[1][0]:None,valideqs[2][0]:None,
    valideqs[3][0]:None,valideqs[0][1]:None,valideqs[1][2]:None,
    valideqs[2][1]:None}
    if not populate(A[0], A1, Lov): return False

    A2 = np.rot90(A2, 1) # rotating into position as A1
    if not populate(A[1], A2, Lov): return False # populating rotated matrix
    A2 = np.rot90(A2, -1) # rotating back

    A3 = np.rot90(A3, -1) # position
    if not populate(A[2], A3, Lov): return False # population
    A3 = np.rot90(A3, 1) # back

    A4 = np.rot90(A4, 2) # position
    if not populate(A[3], A4, Lov): return False # population
    A4 = np.rot90(A4, -2) # back
    print('A4')
    # all matricies check out, add all values to list
    validsolution = []
    
    # adding row 1
    for i in range(3):
        validsolution.append(A2[0,i])

    # adding row 2
    for i in range(2): 
        validsolution.append(A2[1,i])
    for i in range(3):
        validsolution.append(A4[0,i])

    # adding row 3 
    for i in range(3):
        validsolution.append(A1[0,i])
    for i in range(3):
        validsolution.append(A4[1,i])
    
    # adding row 4
    for i in range(3):
        validsolution.append(A1[1,i])
    for i in range(3):
        validsolution.append(A4[2,i])

    # adding row 5
    for i in range(2):
        validsolution.append(A1[2,i])
    for i in range(3):
        validsolution.append(A3[1,i])
    
    # adding row 6
    for i in range(3):
        validsolution.append(A3[2,i])
    
    # adding all the values
    fsum = sum(validsolution)

    # if the final sum is less than 1111, save it as a .txt with the sum as the filename
    with open(f'{fsum}.txt', 'w') as outfile:
        for sol in validsolution:
            outfile.write(f"{sol},")
    
    return True


In [516]:
def fillsquare(A,M,coord1,coord2,pos,Lov):
    # calculating the value for the square
    V = int( A - (M[coord1[0],coord1[1]]+M[coord2[0],coord2[1]]) )
    # checking if the value is in range and has not been repeated elsewhere
    if 0 < V < A and V not in Lov:
        M[pos[0], pos[1]] = V
        Lov[V] = None
        return True
    
    # otherwise return false
    return False

In [517]:
def fillsquaresubroutine(A,M,inputs,Lov,inc):
    for v in range(len(inputs)):
        if not fillsquare(A,M,inputs[v][0],inputs[v][1],inputs[v][2],Lov):
            if not fillsquare(A+inc,M,inputs[v][0],inputs[v][1],inputs[v][2],Lov):
                return False
        else: continue
    return True

In [518]:
# testing fillsquare
# inputs [(val1, val2, target square)]
inputs = [[(0,1),(0,2),(0,0)],
        [(0,0),(2,2),(1,1)],
        [(1,1),(1,2),(1,0)],
        [(0,1),(1,1),(2,1)],
        [(1,1),(0,2),(2,0)]]

# of initial attempts
A = 27 
Lov = {}

attempts = 0
increment = 1
while attempts < 2:
    M = np.array([[0,3,12],
                [0,0,9],
                [0,0,5]])
    if not fillsquaresubroutine(A,M,inputs,Lov,increment):
        attempts += 1
        increment *= -1
        Lov = {}
    else:
        print('Winner!') 
        display(M)
        break

Winner!


array([[12,  3, 12],
       [ 8, 10,  9],
       [ 5, 14,  5]])

In [519]:
def populate(A,M,Lov):
    # creating list of inputs
    # inputs [(val1, val2, target square)]
    inputs = [[(0,1),(0,2),(0,0)],
            [(0,0),(2,2),(1,1)],
            [(1,1),(1,2),(1,0)],
            [(0,1),(1,1),(2,1)],
            [(1,1),(0,2),(2,0)]]
    
    # of initial attempts and initial increment amount
    attempts = 0
    increment = 1
    # checking for magic squares
    while attempts < 2:
        # if filling squares failes, attempt different A +/- 1 
        if not fillsquaresubroutine(A,M,inputs,Lov,increment):
            attempts += 1
            increment *= -1
            Lov = {}
        else: break
    # validating all combinations follow A, or A +/-1
    # checking if columns follow addition rule
    for n in range(3):
        val = M[:,n].sum()
        if val == A or val == A + increment:
            continue
        else: 
            return False
    
    # checking if rows follow addition rule
    for m in range(3):
        val = M[m,:].sum()
        if val == A or val == A + increment:
            continue
        else: 
            return False
        
    # checking if diagonals follow addition rule
    if M.trace() != A and M.trace() != A + increment:
        return False
    if np.rot90(M).trace() != A and np.rot90(M).trace() != A + increment:
        return False
    
    # if all works out, then matrix is valid
    else: return True

In [ ]:
import itertools
import numpy as np

# program takes baseline 3m 22s to run
i = 0
attempted = {}
for A in itertools.combinations(range(80,364), 4):
    # running the permutation engine for each combination of A's
    permutationengine(A, attempted)
    
    # keeping track of the number of combinations
    i += 1
    if i % 100 == 0: print(f"A values processed: {i}, Current A {A}")